# IMPORT

In [2]:
import torch
import glob
import os
import numpy as np
from icecream import  ic
from transformers import AutoTokenizer, AutoModel
from torchvision.transforms.functional import InterpolationMode
from torchvision import transforms
from PIL import Image
import sys

sys.path.append("/data2/npl/ViInfographicCaps/contest/AIC/VideoFrameRetrieval")
from utils.transform import Transform


In [3]:
def get_all_image_path(root):
    all_frames = glob.glob(os.path.join(root, "**", "*.jpg"), recursive=True)
    return all_frames

## LOAD IMAGE TEST

In [4]:
frames_dir = "/data2/npl/ViInfographicCaps/contest/AIC/data/frames/keyframes"
list_frame_of_video_dir = [os.path.join(frames_dir, video_id) for video_id in os.listdir(frames_dir)]

In [34]:
# test_dir = list_frame_of_video_dir[0]
all_test_image_paths = get_all_image_path(frames_dir)

In [35]:
len(all_test_image_paths)

161695

In [6]:
images = [
    Image.open(image_path)
    for image_path in all_test_image_paths[:4]
]

## LOAD MODEL AND RUN

# TEST CODE VERSION 2

In [ ]:
import math
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, AutoConfig

# If you set `load_in_8bit=True`, you will need two 80GB GPUs.
# If you set `load_in_8bit=False`, you will need at least three 80GB GPUs.
path = '/data2/npl/ViInfographicCaps/model/InternVL3-8B'
# device_map = split_model(path)
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=False,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
).to("cuda:3").eval()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

FlashAttention2 is not installed.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [22]:
# set the max number of tiles in `max_num`
pixel_values_1 = load_image(all_test_image_paths[0], max_num=12).to(torch.bfloat16).to("cuda:3")
pixel_values_2 = load_image(all_test_image_paths[1], max_num=12).to(torch.bfloat16).to("cuda:3")
pixel_values = torch.cat([pixel_values_1, pixel_values_2], dim=0)
generation_config = dict(max_new_tokens=1024, do_sample=True)
num_patches_list = [pixel_values_1.size(0), pixel_values_2.size(0)]

# response, history = model.chat(tokenizer, pixel_values, question, generation_config,
#                                num_patches_list=num_patches_list,
#                                history=None, return_history=True)


In [ ]:
# pixel_values1 = load_image('./examples/image1.jpg', max_num=12).to(torch.bfloat16).cuda()
# pixel_values2 = load_image('./examples/image2.jpg', max_num=12).to(torch.bfloat16).cuda()
# pixel_values = torch.cat((pixel_values1, pixel_values2), dim=0)
# num_patches_list = [pixel_values1.size(0), pixel_values2.size(0)]

# question = 'Image-1: <image>\nImage-2: <image>\nDescribe the two images in detail.'
# response, history = model.chat(tokenizer, pixel_values, question, generation_config,
#                                num_patches_list=num_patches_list,
#                                history=None, return_history=True)
# print(f'User: {question}\nAssistant: {response}')

In [ ]:
prefix_template = "Image-{id}: <image>\n"
prefix = "".join([prefix_template.format(id=i+1) for i in range(2)])
# question = '<image>\nPlease describe the image shortly.'
prompt = prefix + """
    Carefully analyze the image below and describe everything you can see in the image in as much detail as possible. 
    Describe all visible objects and people and their actions, positions, and relationships.
    Mention colors, location, and time of day of the scene in the images.

    ### Captions return in the following format:
        Image 1: <Caption 1>
        ...
        Image n: <Caption n>
"""


In [30]:
response = model.chat(tokenizer, pixel_values, prompt, generation_config, num_patches_list=num_patches_list)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


In [31]:
response

'Image-1: The image shows shows an indoor scene where a person in a green shirt is holding a clear plastic bag filled with various items, likely gifts or small boxes, while another person in the background holds a yellow object. The setting appears to be a store or a gift shop, as indicated by the presence of a Christmas tree with decorations and shelves stocked with items in the background. The blurred faces suggest movement or a low-resolution capture. The watermark "tuoi tre TV" is visible in the top right corner.\n\nImage-2: In the image, a person wearing a bright green shirt and blue jeans is holding a box labeled "HANOMO" in a public area, possibly a street or transit area, with a white bus visible in the background. The individual appears to be interacting with another person who is partially visible and wearing a black top. The background includes a shop with a yellow sign that reads "HANOMO" and various items such as water bottles displayed outside. The setting suggests an urb

In [32]:
import re
parts = re.split(r'Image-\d+\s*', response)

In [21]:
parts.remove("")

ValueError: list.remove(x): x not in list

In [33]:
parts

['',
 ': The image shows shows an indoor scene where a person in a green shirt is holding a clear plastic bag filled with various items, likely gifts or small boxes, while another person in the background holds a yellow object. The setting appears to be a store or a gift shop, as indicated by the presence of a Christmas tree with decorations and shelves stocked with items in the background. The blurred faces suggest movement or a low-resolution capture. The watermark "tuoi tre TV" is visible in the top right corner.\n\n',
 ': In the image, a person wearing a bright green shirt and blue jeans is holding a box labeled "HANOMO" in a public area, possibly a street or transit area, with a white bus visible in the background. The individual appears to be interacting with another person who is partially visible and wearing a black top. The background includes a shop with a yellow sign that reads "HANOMO" and various items such as water bottles displayed outside. The setting suggests an urban 